In [2]:
from transformers import RobertaTokenizerFast, AutoModelForTokenClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss, accuracy_score, matthews_corrcoef
from sklearn.metrics import roc_auc_score
from scipy.special import softmax
import pandas as pd
import pickle
import numpy as np
import torch
# import pickle5 as pickle

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
tokenizer = RobertaTokenizerFast.from_pretrained("../checkpoint-final/")
best_ft_model = AutoModelForTokenClassification.from_pretrained("../Outputs/Checkpoints/checkpoint-best/")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
df_test = pd.read_pickle('../Datasets/caid.pkl')


In [7]:
df_test

,ID,Sequence,full
0,DP00084,MSDNDDIEVESDEEQPRFQSAADKRAHHNALERKRRDHIKDSFHSL...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,DP00182,MAPTKRKGSCPGAAPKKPKEPVQVPKLVIKGGIEVLGVKTGVDSFT...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,DP00206,MKAAQKGFTLIELMIVVAIIGILAAIAIPAYQDYTARAQLSERMTL...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,DP00334,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,DP00359,MMLTKSVVISRPAVRPVSTRRAVVVRASGQPAVDLNKKVQDAVKEA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
647,DP02330,MWLPLTVLLLAGIVSADYDHGWHVNNEYIYLVRSRTLVNLNELSDQ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
648,DP02331,MWCPLFLVLLAGAATAEHLQAWKTDTEYQYAVRGRTLSALHDVADQ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
649,DP02332,MAGELADKKDRDASPSKEERKRSRTPDRERDRDRDRKSSPSKDRKR...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
650,DP02333,MSHIQIPPGLTELLQGYTVEVLRQQPPDLVEFAVEYFTRLREARAP...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
def precision_recall_f1_roc_convolve(name, logits, labels, convolution):
    logits = [i[1] for i in logits]
    convolved = np.convolve(np.array(logits).flatten(), np.array(convolution / np.sum(convolution).flatten()), 'same')
    p = [(1 - i, i) for i in convolved]
    roc = [i[1] for i in p]
    print(len(p))
    p = np.argmax(p, axis=-1)
    precision, recall, f1, support = precision_recall_fscore_support(labels, p)
    roc_auc = roc_auc_score(labels, roc)
    mcc = matthews_corrcoef(labels, p)
    return {
        f'precision_{name}':precision[1],
        f'recall_{name}':recall[1],
        f'f1_{name}':f1[1],
        f'roc_auc_{name}':roc_auc,
        f'MCC_{name}':mcc,
    }

In [22]:
def compute_metrics(logits, labels):
    logits = softmax(logits, axis=1)
    l = labels
#     l = []
#     for j, i in enumerate(labels):
#         l = l + list(i)
#     print(len(l), len(logits))
    lg2 = logits
    metrics = {}
    metrics.update(precision_recall_f1_roc_convolve('normal', lg2, l, [1]))
#     metrics.update(precision_recall_f1_roc_convolve('wa5', lg2, l, [1,1,1,1,1]))
#     metrics.update(precision_recall_f1_roc_convolve('wa9', lg2, l, [1,1,1,1,1,1,1]))
#     metrics.update(precision_recall_f1_roc_convolve('wa15', lg2, l, [1]*15))
#     metrics.update(precision_recall_f1_roc_convolve('linear5', lg2, l, [1,2,3,2,1]))
#     metrics.update(precision_recall_f1_roc_convolve('linear9', lg2, l, [1,2,3,4,5,4,3,2,1]))
#     metrics.update(precision_recall_f1_roc_convolve('linear15', lg2, l, [1,2,3,4,5,6,7,8,7,6,5,4,3,2,1]))
#     metrics.update(precision_recall_f1_roc_convolve('quad5', lg2, l, [1,3,9,3,1]))
#     metrics.update(precision_recall_f1_roc_convolve('quad9', lg2, l, [1,3,9,27,81,27,9,3,1]))
#     metrics.update(precision_recall_f1_roc_convolve('quad15', lg2, l, [1,3,9,27,81,243,729,2187,729,243,81,27,9,3,1]))
    
    return metrics

In [11]:
def model_out(encoded, model):
    """Given encoding, pass through model to get logits. Returns logits."""
    encoded.to(device)
    with torch.no_grad():
        output = model(**encoded)
    return output

In [12]:
def get_word_vector(sent, tokenizer, model):
    """Returns logits."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt", truncation=True, max_length=1024)
    out = model_out(encoded, model)
    return out

In [16]:
device = torch.device('cuda')
best_ft_model.to(device)

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(35, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Layer

In [18]:
Xs = []
labs = []
for index, row in df_test.iterrows():
    seq = row['Sequence']
    lab = list(row['full'])
    word_embedding = get_word_vector(seq[:min(1024, len(seq))], tokenizer, best_ft_model)
    word_embedding = word_embedding['logits'][0]
    word_embedding = word_embedding[1:-1]
    word_embedding = word_embedding.cpu()
    word_embedding_full = np.squeeze(word_embedding.numpy())
#     print(word_embedding.shape)
    if len(seq) > 2044:
        for i in range(1022, len(seq) - 1022, 1022):
            word_embedding = get_word_vector(seq[i:i+1022], tokenizer, best_ft_model)
            word_embedding = word_embedding['logits'][0]
            word_embedding = word_embedding[1:-1]
            word_embedding = word_embedding.cpu()
            word_embedding = np.squeeze(word_embedding.numpy())
            word_embedding_full = np.concatenate([word_embedding_full, word_embedding], axis=0)
    if len(seq) > 1024:
        word_embedding = get_word_vector(seq[len(seq) - 1022:len(seq)], tokenizer, best_ft_model)
        word_embedding = word_embedding['logits'][0]
        word_embedding = word_embedding[1:-1]
        word_embedding = word_embedding.cpu()
        word_embedding = np.squeeze(word_embedding.numpy())
        overlap_amount = 1022 - (len(seq) % 1022)
        overlap = word_embedding_full[(-1 * overlap_amount):]
        word_embedding_full[(-1 * overlap_amount):] = (overlap + word_embedding[:overlap_amount]) / 2
        end_chunk = word_embedding[overlap_amount:]
        word_embedding_full = np.concatenate([word_embedding_full, end_chunk], axis=0)
#     print(word_embedding_full.shape, len(seq))
    Xs.append(word_embedding_full)
    labs.append(lab[:len(seq)])

In [19]:
Xs = np.concatenate(Xs)
labs = np.concatenate(labs)

In [20]:
labs.shape

(338068,)

In [23]:
Xs.shape

(338068, 2)

In [26]:
metrics = compute_metrics(Xs, labs)

338068


In [27]:
metrics

{'precision_normal': 0.5006209637357178,
 'recall_normal': 0.44072305842049636,
 'f1_normal': 0.46876635332881095,
 'roc_auc_normal': 0.8111190229898141,
 'MCC_normal': 0.3746017942726573}